In [5]:
import pandas as pd
from sklearn.impute import SimpleImputer
from src.data.columns.remove_columns import not_needed_columns
from src.Helpers.helper import NAPercent

# Create a sample DataFrame
df = pd.read_csv("./Quantitative_data.csv", low_memory=False)
columns = not_needed_columns
df = df.drop(columns, inplace=False, axis=1)
len(df.columns.tolist())
df = df[df.select_dtypes(include=['float', 'int', 'float64', 'int64']).columns]
len(df.columns.tolist())
df.columns[df.isnull().all()]

NA = NAPercent(df)
NA['NA Percent']
NA['NA Percent'] = NA['NA Percent'].astype(float)

NA.to_csv("NAPercent.csv")

df_columns = NA[NA['NA Percent'] < 50].index.tolist()

# We can use this for further analysis
df_great = df[df_columns]

df_great.to_csv("dataframe_good.csv")

df_great["cell_length_a"]


0       99.40
1       99.57
2       99.40
3       98.65
4       98.87
        ...  
3130      NaN
3131      NaN
3132      NaN
3133      NaN
3134      NaN
Name: cell_length_a, Length: 3135, dtype: float64

In [10]:
import pandas as pd
from src.data.columns.remove_columns import not_needed_columns

# Create a sample DataFrame
df = pd.read_csv("./Quantitative_data.csv", low_memory=False)
columns = not_needed_columns
df = df.drop(columns, inplace=False, axis=1)

# Group the data based on a partial string match in the 'Product' column
grouped_data = df.groupby('Species')['Count'].sum()

# Display the grouped data
grouped_data.to_csv("group.csv")

KeyError: 'Column not found: Count'

In [15]:
import pandas as pd

# Provided data
data = {
    'Species': [
        0.1, 0.5, 0.4, 0.9, 0.25, 0.25, 'NMR', 'NMR', 'NMR-AC', 'NMR-AC'
    ],
    'Value': [
        10, 20, 30, 40, 50, 60, 70, 80, 90, 100
    ]
}

# Create a DataFrame from the provided data
df = pd.DataFrame(data)

# Separate 'NMR' and 'NMR-AC' into different columns
df['Species_NMR'] = df['Species'].apply(lambda x: x if isinstance(x, str) and 'NMR' in x else None)
df['Species_NMR-AC'] = df['Species'].apply(lambda x: x if isinstance(x, str) and 'NMR-AC' in x else None)

# Convert 'Species' column to numeric values, ignoring non-numeric entries
df['Species'] = pd.to_numeric(df['Species'], errors='coerce')

# Define custom bins for range grouping
bins = [0, 0.3, 0.6, 1.0]

# Create a new column 'SpeciesRange' based on the range of 'Species' values
df['SpeciesRange'] = pd.cut(df['Species'], bins=bins)

# # Group by 'SpeciesRange' and sum the 'Value' for each range
# grouped_by_range = df.groupby('SpeciesRange')['Value'].sum()

# # Group by 'Species_NMR' and sum the 'Value' for each 'NMR' group
# grouped_by_NMR = df.groupby('Species_NMR')['Value'].sum()

# # Group by 'Species_NMR-AC' and sum the 'Value' for each 'NMR-AC' group
# grouped_by_NMR_AC = df.groupby('Species_NMR-AC')['Value'].sum()

# # Display the grouped data
# print("Grouped by Range:")
# print(grouped_by_range)
# print("\nGrouped by NMR:")
# print(grouped_by_NMR)
# print("\nGrouped by NMR-AC:")
# print(grouped_by_NMR_AC)

df


,Species,Value,SpeciesRange
0,0.10,10,"(0.0, 0.3]"
1,0.50,20,"(0.3, 0.6]"
2,0.40,30,"(0.3, 0.6]"
3,0.90,40,"(0.6, 1.0]"
4,0.25,50,"(0.0, 0.3]"
5,0.25,60,"(0.0, 0.3]"
6,NaN,70,NaN
7,NaN,80,NaN
8,NaN,90,NaN
9,NaN,100,NaN


In [16]:
import pandas as pd
import numpy as np

# Provided data
data = {
    'Species': [
        0.1, 0.5, 0.4, 0.9, 0.25, 0.25, 'NMR', 'NMR', 'NMR-AC', 'NMR-AC'
    ],
    'Value': [
        10, 20, 30, 40, 50, 60, 70, 80, 90, 100
    ]
}

# Create a DataFrame from the provided data
df = pd.DataFrame(data)

# Function to identify NMR or NMR-AC
def get_nmr_type(species):
    if 'NMR' in str(species):
        return 'NMR'
    return species

# Apply the function to create a new column 'NMR_Type'
df['NMR_Type'] = df['Species'].apply(get_nmr_type)

# Convert numeric entries to numeric values
df['NMR_Type'] = pd.to_numeric(df['NMR_Type'], errors='coerce')

# Define custom bins for range grouping
bins = [0, 0.3, 0.6, 1.0, np.inf]
labels = ['0-0.3', '0.3-0.6', '0.6-1.0', 'NMR/NMR-AC']

# Create a new column 'SpeciesRange' based on the range of 'NMR_Type' values
df['SpeciesRange'] = pd.cut(df['NMR_Type'], bins=bins, labels=labels, right=False)

# Group by 'SpeciesRange' and sum the 'Value' for each range
grouped_data = df.groupby('SpeciesRange')['Value'].sum()

# Display the grouped data
print(grouped_data)


SpeciesRange
0-0.3         120
0.3-0.6        50
0.6-1.0        40
NMR/NMR-AC      0
Name: Value, dtype: int64


In [8]:
import pandas as pd
import numpy as np
import html
from src.data.columns.remove_columns import not_needed_columns
from src.Helpers.helper import generate_range_bins, generate_list_with_difference, convert_to_numeric_or_str

# Provided data
df = pd.read_csv("./Quantitative_data.csv", low_memory=False)
columns = not_needed_columns
df = df.drop(columns, inplace=False, axis=1)
# Apply the custom function to 'Column1'
df['Resolution'] = df['Resolution'].apply(convert_to_numeric_or_str)
# Separate 'NMR' and 'NMR-AC' from the 'Species' column
mask_str = df['Resolution'].apply(lambda x: isinstance(x, str))
df_numeric = df[~mask_str]
df_nmr = df[mask_str]

# Convert 'Species' column to numeric values in the numeric DataFrame
df_numeric['Resolution'] = pd.to_numeric(df_numeric['Resolution'], errors='coerce')

# Example usage:
range_resolution_meters = 0.9 # Range resolution of 10 meters
max_range_meters = round(df_numeric['Resolution'].max())     # Maximum range of 1000 meters

print(max_range_meters)

range_bins = generate_range_bins(range_resolution_meters, max_range_meters)

print(len(range_bins))

generated_list = generate_list_with_difference(len(range_bins), range_resolution_meters)
print(len(generated_list))


# Define custom bins for range grouping in the numeric DataFrame
bins = generated_list
labels = range_bins

# Create a new column 'SpeciesRange' based on the range of 'Species' values in the numeric DataFrame
df_numeric['SpeciesRange'] = pd.cut(df_numeric['Resolution'], bins=bins, labels=labels[:-1], right=False)

# Group by 'SpeciesRange' in the numeric DataFrame and sum the 'Value' for each range
grouped_numeric_data = df_numeric.groupby('SpeciesRange').size().reset_index()

# Group by 'Species' in the NMR DataFrame and sum the 'Value' for each species
grouped_nmr_data = df_nmr.groupby('Resolution').size().reset_index()

# Display the grouped data
# print("Numeric Data - Grouped by Range:")
# print(grouped_numeric_data)
# print("\nNMR Data - Grouped by Species:")
# print(grouped_nmr_data)

# Concatenate the dataframes vertically
merged_df = pd.concat([grouped_numeric_data, grouped_nmr_data], ignore_index=True)

# Convert 'Column1' to object data type
merged_df['SpeciesRange'] = merged_df['SpeciesRange'].astype('object')
merged_df['Resolution'] = merged_df['Resolution'].astype('object')

# Update 'Column1' with 'Column2' values where 'Column1' is NaN
merged_df['SpeciesRange'].fillna(merged_df['Resolution'], inplace=True)

# Drop the 'extra if exist' column
merged_df.drop('Resolution', axis=1, inplace=True)

merged_df


40
44
44
44


C:\Users\Awotoroe\AppData\Local\Temp\ipykernel_10636\1147608656.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_numeric['Resolution'] = pd.to_numeric(df_numeric['Resolution'], errors='coerce')
C:\Users\Awotoroe\AppData\Local\Temp\ipykernel_10636\1147608656.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_numeric['SpeciesRange'] = pd.cut(df_numeric['Resolution'], bins=bins, labels=labels[:-1], right=False)


,SpeciesRange,0
0,0.0-0.9,1
1,0.9-1.8,104
2,1.8-2.7,770
3,2.7-3.6,1569
4,3.6-4.5,464
5,4.5-5.4,43
6,5.4-6.3,14
7,6.3-7.2,21
8,7.2-8.1,7
9,8.1-9.0,7


In [7]:
import pandas as pd
import numpy as np
from src.data.columns.remove_columns import not_needed_columns
from src.Helpers.helper import generate_range_bins, generate_list_with_difference, convert_to_numeric_or_str

# Provided data
df = pd.read_csv("./Quantitative_data.csv", low_memory=False)
columns = not_needed_columns
df = df.drop(columns, inplace=False, axis=1)

max_range_meters=40
# Apply the custom function to 'Column1'
df['Resolution'] = df['Resolution'].apply(convert_to_numeric_or_str)
# Example usage:
range_resolution_meters = 0.3 # Range resolution of 10 meters

range_bins = generate_range_bins(range_resolution_meters, max_range_meters)

print(range_bins)

generated_list = generate_list_with_difference(len(range_bins), range_resolution_meters)
print(generated_list)


['0.0-0.3', '0.3-0.6', '0.6-0.9', '0.9-1.2', '1.2-1.5', '1.5-1.8', '1.8-2.1', '2.1-2.4', '2.4-2.7', '2.7-3.0', '3.0-3.3', '3.3-3.6', '3.6-3.9', '3.9-4.2', '4.2-4.5', '4.5-4.8', '4.8-5.1', '5.1-5.4', '5.4-5.7', '5.7-6.0', '6.0-6.3', '6.3-6.6', '6.6-6.9', '6.9-7.2', '7.2-7.5', '7.5-7.8', '7.8-8.1', '8.1-8.4', '8.4-8.7', '8.7-9.0', '9.0-9.3', '9.3-9.6', '9.6-9.9', '9.9-10.2', '10.2-10.5', '10.5-10.8', '10.8-11.1', '11.1-11.4', '11.4-11.7', '11.7-12.0', '12.0-12.3', '12.3-12.6', '12.6-12.9', '12.9-13.2', '13.2-13.5', '13.5-13.8', '13.8-14.1', '14.1-14.4', '14.4-14.7', '14.7-15.0', '15.0-15.3', '15.3-15.6', '15.6-15.9', '15.9-16.2', '16.2-16.5', '16.5-16.8', '16.8-17.1', '17.1-17.4', '17.4-17.7', '17.7-18.0', '18.0-18.3', '18.3-18.6', '18.6-18.9', '18.9-19.2', '19.2-19.5', '19.5-19.8', '19.8-20.1', '20.1-20.4', '20.4-20.7', '20.7-21.0', '21.0-21.3', '21.3-21.6', '21.6-21.9', '21.9-22.2', '22.2-22.5', '22.5-22.8', '22.8-23.1', '23.1-23.4', '23.4-23.7', '23.7-24.0', '24.0-24.3', '24.3-24.6', 

In [13]:
d = [2,3,4,5,2,1,8,5,8,4]
d[:-3]

[2, 3, 4, 5, 2, 1, 8]

In [10]:
import pandas as pd
import altair as alt

# Sample data
data = {
    'Category': ['Category A', 'Category C', 'Category B', 'Category D'],
    'Value': [10, 5, 8, 12]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Define the desired order of categories
desired_order = ['Category A', 'Category B', 'Category C', 'Category D']

# Create the bar chart using Altair
chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Category:N'),
    y=alt.X('Value:Q', sort=desired_order),
)

# Show the chart
chart.show()


Displaying chart at http://localhost:57196/


In [1]:
import pandas as pd
import re

# Sample DataFrame with HTML tags in columns
df = pd.read_csv("../PDB_PROJECT/Executables/enriched_db.csv", low_memory=False)

# Function to remove all HTML tags from a string
def remove_html_tags(text):
    try:
        if not text is None and pd.notna(text):
            clean_text = re.sub(r'<.*?>', '', text)
            # Replace '\r' and '\n' with a space
            clean_text = clean_text.replace('\r', ' ').replace('\n', ' ')
            return clean_text
        else:
            return ''
    
    except (Exception, TypeError) as e:
        print(text)
        print(str(e))

# Filter out columns with string data type
filtered_df = df.select_dtypes(include='object')

filtered_df = filtered_df.applymap(remove_html_tags)

filtered_df


,Group,Subgroup,Pdb Code,Is Master Protein?,Name,Species,Taxonomic Domain,Expressed in Species,Resolution,Description,...,pdbx_serial_crystallography_sample_delivery,rcsb_entry_info.ndb_struct_conf_na_feature_combined,pdbx_serial_crystallography_sample_delivery_injection,pdbx_serial_crystallography_sample_delivery_fixed_target,pdbx_serial_crystallography_data_reduction,pdbx_serial_crystallography_measurement,em_staining,pdbx_database_status.methods_development_category,em2d_crystal_entity,pdbx_nmr_ensemble.torsion_angle_constraint_violation_method
0,MONOTOPIC MEMBRANE PROTEINS,Cyclooxygenases,1PRH,MasterProtein,Ram Prostaglandin H2 synthase-1 (cyclooxygenas...,Ovis aries,Eukaryota,,3.5,,...,,,,,,,,,,
1,MONOTOPIC MEMBRANE PROTEINS,Cyclooxygenases,1PTH,1PRH,Ram Prostaglandin H2 synthase-1 (COX-1),Ovis aries,Eukaryota,,3.4,In complex with bromoaspirin.,...,,,,,,,,,,
2,MONOTOPIC MEMBRANE PROTEINS,Cyclooxygenases,1CQE,1PRH,Ram Prostaglandin H2 synthase-1 (COX-1),Ovis aries,Eukaryota,,3.1,In complex with flurbiprofen.,...,,,,,,,,,,
3,MONOTOPIC MEMBRANE PROTEINS,Cyclooxygenases,1EQG,1PRH,Ram Prostaglandin H2 synthase-1 (COX-1),Ovis aries,Eukaryota,,2.61,1EQG is complex with ibuprofen. Complex with ...,...,,,,,,,,,,
4,MONOTOPIC MEMBRANE PROTEINS,Cyclooxygenases,1EBV,1PRH,Ram Prostaglandin H2 synthase-1 (COX-1),Ovis aries,Eukaryota,,3.2,In complex with O-actylsalicylhydroxamic acid.,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3130,TRANSMEMBRANE PROTEINS: ALPHA-HELICAL,Light-Harvesting+Reaction Center Complexes,7F0L,MasterProtein,LH1-RC super complex,Rhodobacter sphaeroides,Bacteria,,2.94,cryo-EM structure,...,,,,,,,,,,
3131,TRANSMEMBRANE PROTEINS: ALPHA-HELICAL,Light-Harvesting+Reaction Center Complexes,7PIL,7F0L,RC-LH1-PufXY monomer complex,Rhodobacter sphaeroides,Bacteria,,2.50,cryo-EM structure,...,,,,,,,,,,
3132,TRANSMEMBRANE PROTEINS: ALPHA-HELICAL,Light-Harvesting+Reaction Center Complexes,7VY2,7F0L,LH1-RC super complex dimer,Cereibacter sphaeroides,Bacteria,,2.75,"cryo-EM structure structure lacking protein-U,...",...,,,,,,,,,,
3133,TRANSMEMBRANE PROTEINS: ALPHA-HELICAL,Light-Harvesting+Reaction Center Complexes,7VOR,7F0L,"LH1-RC super complex dimer, in class 1",Cereibacter sphaeroides,Bacteria,,2.74,"cryo-EM structure in class-2, 2.90 Å 7VOT PufX...",...,,,,,,,,,,


In [5]:
filtered_df['Resolution'].describe

<bound method NDFrame.describe of 0        3.5
1        3.4
2        3.1
3       2.61
4        3.2
        ... 
3130    2.94
3131    2.50
3132    2.75
3133    2.74
3134    2.50
Name: Resolution, Length: 3135, dtype: object>

In [ ]:
import pandas as pd
import re
from src.implementation.visualization import DataImport

# Sample DataFrame with HTML tags in columns
# df = pd.read_csv("./dist/data_folder/enriched_db.csv", low_memory=False)
data_import = DataImport()
dataset = data_import.loadFile()